In [1]:
#mnist data overview
#600000 fotoğraflı kütüphane 4 dimensions  28 by 28 pixels
#(60000,28,28,1) (samples,x,y,channels) gray olduğundan sadece tek channel var
#bu demek oluyor orginal labeller bir sayı listesi olarak verilioyr
#[5,0,4,....,5,6,8] gibi
#bunları one hot encodinge dönüştüreceğiz(keras ile kolay bi şkeilde)
#one hot encoding bir listede hangi resim o resmi temsil ediyorsa ona 1 yazıldığı bir modeş
#label is represented based of index position in the label array
#the corresponding label will be a 1 at the index location and zero everwhere else
#example digit 4 would have this label array
#[0,0,0,0,1,0,0,0,0,0]  index locationu 0,1,2,3,4 olduğundan 5 inci bölgede 1 var

In [2]:
#CONVOLUTİONAL NEURAL NETWORKS

In [ ]:
#each unit connected to smaller unit
#3by 3 filter - multiply by filter weights - sum the result- output (convolution)
#after convolution you do subsampling pooling 
#pooling layers will subsample the input image , which reduces memory use as well as
#reducing the number of parameters
#2 by 2 kernel take and evaluate the max value
# sonraki layera geçen değer bu max değer olucak misal 4 kare içinde en büyük değer 4 <
#o zaman o 4 değeri diğer layera inerken gelicek ,


#DROPOUT TEKNİĞİ

#Regularize etme süreci overfittingi engellemek için
#training esnasında unitler rastlege droplanıyor bağlantılarıyla beraber
#co adaptasyonu engelliyor buda
#